In [ ]:
!python -m pip install -r requirements.txt

In [ ]:
import numpy as np
from scipy.integrate import odeint

def lotka_volterra_model(y: np.ndarray, t: float,
        alpha: float, beta: float, gamma: float, delta: float) -> np.ndarray:
    u, v = y
    return np.array([(alpha - beta * v) * u, (-gamma + delta * u) * v])

def ode(y: np.ndarray, t: float,
        alpha: float, beta: float, gamma: float, delta: float) -> np.ndarray:
    (u, v, u_alpha, u_beta, u_gamma, u_delta, v_alpha, v_beta, v_gamma, v_delta
    ) = y
    return np.array([
        (alpha - beta * v) * u,
        (-gamma + delta * u) * v,

        u + (alpha - beta * v) * u_alpha + (-beta * u) * v_alpha,
        -v * u + (alpha - beta * v) * u_beta + (-beta * u) * v_beta,
        (alpha - beta * v) * u_gamma + (-beta * u) * v_gamma,
        (alpha - beta * v) * u_delta + (-beta * u) * v_delta,

        delta * v * u_alpha + (-gamma + delta * u) * v_alpha,
        delta * v * u_beta + (-gamma + delta * u) * v_beta,
        -v + delta * v * u_gamma + (-gamma + delta * u) * v_gamma,
        u * v + delta * v * u_delta + (-gamma + delta * u) * v_delta
    ])

def func(x: np.ndarray, t: np.ndarray, u: np.ndarray, v: np.ndarray) -> float:
    y = odeint(lotka_volterra_model, np.array([u[0], v[0]]), t, args=tuple(x))
    return np.sum(np.power(y[:, 0] - u, 2)) + np.sum(np.power(y[:, 1] - v, 2))

def func_jac(x: np.ndarray, t: np.ndarray,
        u: np.ndarray, v: np.ndarray) -> np.ndarray:
    n = len(x)
    y0 = np.zeros(2 + 2 * n)
    y0[:2] = u[0], v[0]
    y = odeint(ode, y0, t, args=tuple(x)).T
    result = np.zeros(n)
    for i in range(n):
        result[i] += np.sum((y[0] - u) * y[2 + i])
        result[i] += np.sum((y[1] - v) * y[2 + n + i])
    result *= 2.0
    return result

In [ ]:
start, stop, n = 0.0, 5.0, 2 ** 10 + 1
t = np.linspace(start, stop, n)

In [ ]:
import matplotlib.pyplot as plt

y0 = np.array([30.0, 10.0])
args = 10, 1.0, 10, 0.5
u, v = odeint(lotka_volterra_model, y0, t, args=args).T

fig = plt.figure(figsize=(16, 9), dpi=400)
subplot = fig.add_subplot(111, facecolor='white', title='Predator-Prey',
    xlabel='Time', ylabel='Population')
subplot.plot(t, v, 'orange', label='predator', linewidth=2)
subplot.plot(t, u, 'blue', label='prey', linewidth=2)
plt.legend()
plt.show()

In [ ]:
fig = plt.figure(figsize=(16, 9), dpi=400)
subplot = fig.add_subplot(111, facecolor='white', title='Phase portrait',
    xlabel='Prey', ylabel='Predator')
subplot.plot(u, v, 'blue', label='predator-prey', linewidth=2)
plt.legend()
plt.show()

In [ ]:
from numpy.random import normal

scale = 0.05
u_noise = u + u * normal(loc=0.0, scale=scale, size=n)
v_noise = v + v * normal(loc=0.0, scale=scale, size=n)

In [ ]:
fig = plt.figure(figsize=(16, 9), dpi=400)
subplot = fig.add_subplot(111, facecolor='white', title='Predator-Prey',
    xlabel='Time', ylabel='Population')
subplot.plot(t, v, 'orange', label='predator', linewidth=2)
subplot.plot(t, u, 'blue', label='prey', linewidth=2)
subplot.plot(t, v_noise, 'Xr', label='predator_noise')
subplot.plot(t, u_noise, 'Xg', label='prey_noise')
plt.legend()
plt.show()

In [ ]:
fig = plt.figure(figsize=(16, 9), dpi=400)
subplot = fig.add_subplot(111, facecolor='white', title='Phase portrait',
    xlabel='Prey', ylabel='Predator')
subplot.plot(u, v, 'blue', label='predator-prey', linewidth=2)
subplot.plot(u_noise, v_noise, 'Xg', label='predator-prey noise')
plt.legend()
plt.show()

In [ ]:
from scipy.optimize import minimize

y0_noise, tol, max_iter = np.array([u_noise[0], v_noise[0]]), 1E-7, 10000

x0, delta, initial_simplex = np.zeros(4), 0.15, np.zeros((5,4))
initial_simplex[1:] = 2.0 * delta * np.identity(4)
initial_simplex = (initial_simplex + (1.0 - delta)) * args
result = minimize(fun=func, x0=x0, args=(t, u_noise, v_noise),
    method='Nelder-Mead', tol=tol,
    options={'initial_simplex': initial_simplex, 'maxiter': max_iter})
print(F'The solution of the optimization: {result.x}')
print(F'Value of objective function: {result.fun}')
print(F'Description of the cause of the termination: {result.message}')
print(F'Number of evaluations of the objective functions: {result.nfev}')
print(F'Number of iterations performed by the optimizer: {result.nit}')

u_approx, v_approx = odeint(lotka_volterra_model, y0_noise, t,
    args=tuple(result.x)).T

fig = plt.figure(figsize=(16, 9), dpi=400)
subplot = fig.add_subplot(111, facecolor='white', title='Predator-Prey',
    xlabel='Time', ylabel='Population')
subplot.plot(t, v_noise, 'Xr', label='predator_noise')
subplot.plot(t, v, 'orange', label='predator', linewidth=2)
subplot.plot(t, v_approx, 'c--', label='predator_approx', linewidth=2)
subplot.plot(t, u_noise, 'Xg', label='prey_noise')
subplot.plot(t, u, 'blue', label='prey', linewidth=2)
subplot.plot(t, u_approx, 'm--', label='prey_approx', linewidth=2)
plt.legend()
plt.show()

In [ ]:
x0=(1.0 + normal(loc=0.0, scale=0.4, size=4)) * args
result = minimize(fun=func, x0=x0, args=(t, u_noise, v_noise),
    method='TNC', jac=func_jac, tol=tol, options={'maxiter': max_iter})
print(F'The solution of the optimization: {result.x}')
print(F'Value of objective function: {result.fun}')
print(F'Description of the cause of the termination: {result.message}')
print(F'Number of evaluations of the objective functions: {result.nfev}')
print(F'Number of iterations performed by the optimizer: {result.nit}')

u_approx, v_approx = odeint(lotka_volterra_model, y0_noise, t,
    args=tuple(result.x)).T

fig = plt.figure(figsize=(16, 9), dpi=400)
subplot = fig.add_subplot(111, facecolor='white', title='Predator-Prey',
    xlabel='Time', ylabel='Population')
subplot.plot(t, v_noise, 'Xr', label='predator_noise')
subplot.plot(t, v, 'orange', label='predator', linewidth=2)
subplot.plot(t, v_approx, 'c--', label='predator_approx', linewidth=2)
subplot.plot(t, u_noise, 'Xg', label='prey_noise')
subplot.plot(t, u, 'blue', label='prey', linewidth=2)
subplot.plot(t, u_approx, 'm--', label='prey_approx', linewidth=2)
plt.legend()
plt.show()